# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

##### Students

* Team: `44`
* Students: `<Silvan Stettler, Diego Debruyn, Yann Morize, Nikolai Orgland>`
* Dataset: `<the dataset you used to complete the milestone>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [ ]:
# Your code here.
data = pd.read_csv('../data/abdb-de-all-affairs-50-0_new.csv', sep=',',lineterminator='\n',
                   encoding='utf-8',engine='c', low_memory=False)
members_cn = pd.read_csv('../data/conseil_national_members.csv')

# Need to keep: 
data = data[['AffairShortId','AffairTitle','VoteDate','CouncillorId','CouncillorName',
            'CouncillorYes','CouncillorNo','CouncillorAbstain','CouncillorNotParticipated',
            'CouncillorExcused','CouncillorPresident']]

data[['VoteDate']] = data[['VoteDate']].apply(lambda x: x.str.slice(4,15))
#features = data.loc[:, []] # the pandas dataframe with the features and labels

In [ ]:
# Only keep data from councillors that are in office today
data = data[data.CouncillorName.isin(members_cn['Name'])]


# Delete all votes concerning "Ordnungsanträge"
data = data[~((data.AffairShortId == 1) | (data.AffairShortId == 2))]

# Create list of nodes containing name and Id number of councillor
nodes = data[['CouncillorId','CouncillorName']].drop_duplicates(keep='first')

# List of all affairs with their Id number. The votes of one affair corresponds to one feature of the nodes
affairs = data[['AffairShortId','AffairTitle']].drop_duplicates(keep='first')
#affairs = affairs.rename(columns={'index':'feature_idx'})
#affairs.shape
nodes.shape


In [ ]:
# A feature of a node is equal to the vote concerning a certain affair (-> one affair_id represents one feature)
# Replace the Affair Ids by a new index (feature index)
affairid2feature = affairs[['AffairShortId']]
affairid2feature.insert(1,'feature_idx',np.arange(1,affairid2feature.shape[0]+1))
affairid2feature = affairid2feature.set_index('AffairShortId')

In [ ]:
from create_network import *

# Convert 'Yes','No','Abstain' etc. to a numerical value
val_votes = assign_val_to_vote(data)
val_votes = val_votes[['CouncillorId','AffairShortId', 'value']]
val_votes = val_votes.join(affairid2feature, on='AffairShortId')
val_votes = val_votes.drop(columns='AffairShortId')

In [ ]:
# Reduce memory usage
val_votes.loc[:,'CouncillorId'] = downcast_column(val_votes, 'CouncillorId','integer')
val_votes.loc[:,'value'] = downcast_column(val_votes, 'value','float')
val_votes.loc[:,'feature_idx'] = downcast_column(val_votes, 'feature_idx','integer')
val_votes = val_votes.set_index('feature_idx')

In [ ]:
# These df containing the complete feature_idx are
complete_feat_idx = affairid2feature[['feature_idx']]
complete_feat_idx = complete_feat_idx.set_index('feature_idx')

# Start the clean dataset with just an index column
data_clean = complete_feat_idx

for councId in nodes.loc[:,'CouncillorId']:
    feature_vec = extract_feature_vec(val_votes, councId, complete_feat_idx)
    data_clean = pd.concat([data_clean, feature_vec], axis=1,sort=False)


In [ ]:
data_clean.index = data_clean.index.rename('Counc_Id')
features = data_clean.T
features.head()

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
featuresfill=features.fillna(0.5)
featuresfill=featuresfill.reset_index(drop='true')
featuresfill.index.names = ['index']
featuresfill.columns.names = ['vote']
featuresfill.head()

In [ ]:


# Your code here



n_nodes=nodes.shape[0]
distances = np.zeros((n_nodes, n_nodes))


n_nodes=nodes.shape[0]


for i, a in featuresfill.iterrows():
    for j, b in featuresfill.iterrows():
        dot_product=np.dot(a,b)#computation of distance
        distances[i,j]=1-dot_product/(np.linalg.norm(a,2)*np.linalg.norm(b,2))
        

kernel_width = distances.mean()
weights = np.exp(-distances**2 / kernel_width**2)
weights[:5,:5]


adjacency=weights.copy()

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
plt.spy(adjacency, markersize=0.050000)
plt.title('adjacency matrix')
plt.imshow(weights, cmap=plt.cm.Blues);
plt.colorbar();
plt.show()

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
# Your code here.
Lmax=n_nodes*(n_nodes-1)/2
L=(np.count_nonzero(adjacency)-n_nodes)/2
print(L/Lmax)


**Your answer here.**

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**Your answer here.**

In [ ]:
# Your code here.

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
# Your code here.
np.save('adjacency',adjacency)
features.to_csv('paunocu.csv', encoding='utf-8')

### Question 4

Are the edges of your graph weighted?

**Yes, they are**

### Question 5

What is the degree distibution of your network? 

In [ ]:
#degree =np.count.nonzeros(adjacency,axis=0)  # Your code here. It should be a numpy array.
degree=np.count_nonzero(adjacency,axis=0)
assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
# Your code here.
average=degree.mean()
print(average)

### Question 6

Comment on the degree distribution of your network.

**Your answer here.**

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    input_adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    
    nb_nodes = list(adjacency.shape)[0] 
    # gives us number of nodes of the adjacency matrix
    visited_nodes = []
    # here we save the nodes we have visited
    waiting_nodes = [0]
    # here we save the new neighbouring in a waiting list
    
    # the while loop visits the new neighbouring nodes until all are visited of the connected part
    while waiting_nodes:
        # pop shallowest node (first node) from queue     
        node = waiting_nodes.pop(0)
        # we select the first node of the waiting list 
        if node not in visited_nodes:
            visited_nodes.append(node)
            # we add the node as visited
            neighbours_array = np.nonzero(adjacency[node,:])
            # we get tthe neighbours of the node
            neighbours = list(neighbours_array)
            neighbours = neighbours[0]
 
            # add neighbours of node to queue (as a list)
            for neighbour in neighbours:
                waiting_nodes.append(neighbour)  
            # we add the new neighbour nodes to the waiting list     
                
    nb_nodes_found = len(visited_nodes)
    # we check if we visited all nodes
    
    if nb_nodes_found == nb_nodes:
        connected =  True
    # if we visited all nodes, the network is connected
    else:
        connected = False
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
connected_graph(adjacency)

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    nb_nodes = list(adjacency.shape)[0] 
    # gives us number of nodes of the adjacency matrix
    visited_nodes = []
    # here we save the nodes we have visited
    waiting_nodes = [0]
    # here we save the new neighbouring in a waiting list
    
    # the while loop visits the new neighbouring nodes until all are visited of the connected part
    while waiting_nodes:
        # pop shallowest node (first node) from queue     
        node = waiting_nodes.pop(0)
        # we select the first node of the waiting list 
        if node not in visited_nodes:
            visited_nodes.append(node)
            # we add the node as visited
            neighbours_array = np.nonzero(adjacency[node,:])
            # we get tthe neighbours of the node
            neighbours = list(neighbours_array)
            neighbours = neighbours[0]
 
            # add neighbours of node to queue (as a list)
            for neighbour in neighbours:
                waiting_nodes.append(neighbour)  
            # we add the new neighbour nodes to the waiting list     
                
    components = visited_nodes
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

**Answer:** As we have only one fully connected network, we only have one large connected component with a size of 200 nodes (which corresponds to the number of members of the national parliament). In the case that we have several distinct connected networks: After finding the first connected component, we would have to select one of the not visited nodes and run our find_component function again in order to find the remaining connected components. 

In [ ]:
connected_components = find_components(adjacency)

In [ ]:
size_largest_component = len(connected_components)

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
def compute_shortest_path_lengths(adjacency, source, threshhold):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    
    mask=adjacency>treshhold
    masked=mask.astype(np.int)
    shortest_path_lengths=np.full(n_nodes, -1) #creating distance list
    
    np.fill_diagonal(masked, 0) #unconnecting nodes with themselves
    temp_row=masked[source,:] #adjacency row of starting point
    connected=np.transpose(np.nonzero(temp_row))[:,0] #list of adjacent nodes
    waiting=connected.tolist()       #initializing waiting list    
    np.put(shortest_path_lengths,connected,1)            #initialize distance list
    np.put(shortest_path_lengths,source,0)
    i=0
    
    while (waiting):
        n=shortest_path_lengths[waiting[0]]
        temp_row=masked[waiting[0],:]
        connected=np.transpose(np.nonzero(temp_row))[:,0]
        new_connexions=[]
        for idx in connected:
            if (shortest_path_lengths[idx]==-1):
                new_connexions.append(idx)
                
        np.put(shortest_path_lengths,new_connexions,n+1)
        waiting.extend(new_connexions) 
        waiting.pop(0)
        i=i+1
    return shortest_path_lengths

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
D =compute_shortest_path_lengths(adjacency,0,0.7)

for i in range(n_nodes-1):
    temp=compute_shortest_path_lengths(adjacency,i+1,0.7)
    D=np.vstack((D,temp))

print(D)

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Create binary matrix
    adj = np.zeros(adjacency.shape)
    adj[adjacency > 0] = 1
    
    adj_pow = np.linalg.matrix_power(adj, length)
    n_paths = adj_pow[source,target]
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
def compute_total_paths_in_graph(adjacency, length):
    # Create binary matrix
    adj = np.zeros(adjacency.shape)
    adj[adjacency > 0] = 1
    
    # Raise adjacency to the power of length in order to find all paths of length length 
    adj_pow = np.linalg.matrix_power(adj, length)
    # Only consider upper triangular part of the matrix since we count the path of i to j and j to i as one path
    return np.sum(np.triu(adj_pow,0))

In [ ]:
length = 3
print("The total number of paths of length {0} is {1}".format(length, compute_total_paths_in_graph(adjacency, length)))

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.